# PCA of Sagebrush Promoter Data

In [1]:
using CSV, DataFrames, LinearAlgebra, Plots, Clustering, Distances, Statistics

In [10]:
path = "/Users/jimbeck/Desktop/vip-g2p-buerki-and-melton/Data/Sagebrush_Promoter/"
file = "promoter_catCount_by_Scaff.csv"
cat_count_file₁ = joinpath(path,file)

"/Users/jimbeck/Desktop/vip-g2p-buerki-and-melton/Data/Sagebrush_Promoter/promoter_catCount_by_Scaff.csv"

In [9]:
pwd()

"/Users/jimbeck/Desktop/vip-g2p-buerki-and-melton/Code"

In [11]:
#cat_data = CSV.File(cat_count_file₁; transpose=true, normalizenames=true);
df = DataFrame(CSV.File(file; transpose=true, normalizenames=true))

LoadError: ArgumentError: "promoter_catCount_by_Scaff.csv" is not a valid file

## Put into DataFrame

In [ ]:
df₁ = DataFrame(cat_data);

## Make names for grouping

In [ ]:
rename!(df₁, :water_stress => :σ_water, 
    :temperature_stress => :σ_temp, :stress_hormone => :σ_hormone,:other_stress => :σ_other);

In [ ]:
split_names = split.(df₁[:,1],"_");
number_of_scaffolds = length(split_names)

scaffold_names = []
gene_names = []
gene_type = []
gene_category = []

for i ∈ 1:number_of_scaffolds
    values = split_names[i,:][1]
    push!(scaffold_names, values[1])
    push!(gene_names, values[2])
    push!(gene_category, values[2][1:3])
    if length(values[2]) > 3
        push!(gene_type, values[2][1:4])
    else
        push!(gene_type, values[2][1:3])
    end
end

In [ ]:
df₁.scaffolds = scaffold_names;
df₁.genes = gene_names;
df₁.gene_type = gene_type;
df₁.gene_category = gene_category;

## Basic Information

In [ ]:
names(df₁)

In [ ]:
showall(df₁)

## SVD

### Count Data

## Methods

We implemented a singular value decomposition of the Sagebrush Promoter gene scaffolds to discern their organization according to various principle components within the decomponsition. The first principle component  explains approximatley 72% of the data's variation and is heavily weighted towards light induced stress. The second principle component explains approximately 7.5% of the variation and is weighted towards the ABA stress hormone and water stresses. Categorization of the scaffolds shows the organization of NIPs, PIPs and TIPs to address different combinations of light, ABA and water stresses.

Additionally, we ran a k-means clustering analysis (with k = 9) on the raw promoter data and then overlayed these clusters upon principle components (1 and 2) to visualize clustering of promotors and the clusters relationships to the principle component projections. This overlay seems to reinforce the notion that clusters of different promotors are collaborating to address the stresses driven by each of the principle components - with principle component one primarily motivated by light stress and principle component two primarily related to ABA hormone and water stress.

In [ ]:
matrix₁ = convert(Array{Float64},df₁[:,2:8]);

In [ ]:
F₁ = svd(matrix₁);

In [ ]:
F₁.U;

In [ ]:
F₁.S;

In [ ]:
percent_explained₁ = []
total = sum(F₁.S)
for i ∈ 1:length(F₁.S)
    push!(percent_explained₁,F₁.S[i]/total*100)
end
percent_explained₁

In [ ]:
describe(df₁[:,2:8])

### 𝑉  - Table with columns of Principle Component Recipes 
---
PC1 to PC7 in columns 1 to 7, each with an associated value attributed to the above numbered variables)

For example, PC1 is comprised of 0.789098 parts of the light vector and 0.359209 parts of the ABA vector ...

Since these are vectors in space, we could invert them to use positive values - we are primarily interested in magnitude.

In [ ]:
𝑉 = F₁.V

In [ ]:
# Export CSV file of EigenVectors
CSV.write(joinpath(data_path, "pcRecipe.csv"),DataFrame(𝑉))

## Plotting

In [ ]:
plotlyjs()

In [ ]:
gene_shapes = [:circle :rect :diamond :utriangle :dtriangle]

### Two Dimensional Plot Arrays

In [ ]:
gene_plots₂ = []
type_plots₂ = []
category_plots₂ = []
for i ∈ 1:7, j ∈ 1:7
    if i ≥ j 
        continue
    end

    V₁ = 𝑉[:,i];
    V₂ = 𝑉[:,j];
    a = convert(Matrix,df₁[:,2:8]) * V₁;
    b = convert(Matrix,df₁[:,2:8]) * V₂;
    
    push!(gene_plots₂, scatter(a, b, group = df₁[:,10], 
                                size = (800,800),
                                title ="PC$i and PC$j", 
                                xlabel="PC$i", 
                                ylabel="PC$j", 
                                hover = df₁.ScaffoldID,
                                markersize = 6, 
                                markershape = gene_shapes,
                                legend=:outerright))
    push!(type_plots₂, scatter(a, b, group = df₁[:,11],
                                size = (800,800), 
                                title ="PC$i and PC$j", 
                                xlabel="PC$i", 
                                ylabel="PC$j",
                                hover = df₁.scaffolds,
                                markersize = 6,
                                markershape = gene_shapes,
                                legend=:outerright))
    push!(category_plots₂, scatter(a, b, group = df₁[:,12], 
                                size = (800,800), 
                                title ="PC$i and PC$j",
                                xlabel="PC$i", 
                                ylabel="PC$j",
                                hover = df₁.ScaffoldID,
                                markersize = 6,
                                markershape = gene_shapes,
                                legend=:outerright))
    
end

### Three Dimensional Plot Arrays

In [ ]:
gene_plots₃ = []
type_plots₃ = []
category_plots₃ = []
for i ∈ 1:7, j ∈ 1:7, k ∈ 1:7
    if i ≥ j || j ≥ k 
        continue
    end

    V₁ = 𝑉[:,i];
    V₂ = 𝑉[:,j];
    V₃ = 𝑉[:,k];
    a = convert(Matrix,df₁[:,2:8]) * V₁;
    b = convert(Matrix,df₁[:,2:8]) * V₂;
    c = convert(Matrix,df₁[:,2:8]) * V₃;
    
    push!(gene_plots₃, scatter(a, b, c, 
                                group = df₁[:,10],
                                size = (800,800),
                                title ="PC$i, PC$j and PC$k",
                                xlabel="PC$i", 
                                ylabel="PC$j",
                                zlabel="PC$k", 
                                hover = df₁.ScaffoldID,
                                markersize = 4, 
                                markershape = gene_shapes,
                                legend=:outerright))
    push!(type_plots₃, scatter(a, b, c, 
                                group = df₁[:,11], 
                                size = (800,800),
                                title ="PC$i, PC$j and PC$k",
                                xlabel="PC$i", 
                                ylabel="PC$j",
                                zlabel="PC$k",
                                hover = df₁.scaffolds,
                                markersize = 4,
                                markershape = gene_shapes,
                                legend=:outerright))
    push!(category_plots₃, scatter(a, b, c, 
                                group = df₁[:,12],
                                size = (800,800),
                                title ="PC$i, PC$j and PC$k", 
                                xlabel="PC$i",
                                ylabel="PC$j",
                                zlabel="PC$k", 
                                hover = df₁.ScaffoldID,
                                markersize = 4,
                                markershape = gene_shapes,
                                legend=:outerright))
    
end

## Gene Plots in Two Dimensions

In [ ]:
i=1; # Make Slider

In [ ]:
category_plots₂[i];

In [ ]:
type_plots₂[i];

In [ ]:
gene_plots₂[i];

### Gene Plots in Three Dimensions

In [ ]:
j=1;

In [ ]:
category_plots₃[j];

In [ ]:
type_plots₃[j];

In [ ]:
gene_plots₃[j];

### K-Means

In [ ]:
dmat = pairwise(SqEuclidean(),transpose(matrix₁));
fit = []

for k ∈ 3:28
    cluster = kmeans(transpose(matrix₁),k)
    silResult = mean(silhouettes(cluster,dmat))
    push!(fit, silResult) 
end

In [ ]:
k_count = collect(3:28);
k_fit = hcat(k_count, fit)

In [ ]:
k_cluster₃ = kmeans(transpose(matrix₁),3);
k_cluster₄ = kmeans(transpose(matrix₁),4);
k_cluster₆ = kmeans(transpose(matrix₁),6);
k_cluster₇ = kmeans(transpose(matrix₁),7);

In [ ]:
k_plot₃ = scatter(convert(Matrix,df₁[:,2:8]) * 𝑉[:,1],
    convert(Matrix,df₁[:,2:8]) * 𝑉[:,2],
    marker_z = k_cluster₃.assignments,
    color =:rainbow,
    size=(800,800),
    markersize=8,
    markershape = gene_shapes,
    hover = df₁.ScaffoldID,
    legend=:none,
    title ="PC 1 and PC 2 grouped by k-means clusters, k=3",
    xlabel="PC 1", 
    ylabel="PC 2",
    )

In [ ]:
k_plot₄ = scatter(convert(Matrix,df₁[:,2:8]) * 𝑉[:,1],
    convert(Matrix,df₁[:,2:8]) * 𝑉[:,2],
    marker_z = k_cluster₄.assignments,
    color =:rainbow,
    size=(800,800),
    markersize=8,
    markershape = gene_shapes,
    hover = df₁.ScaffoldID,
    legend=:none,
    title ="PC 1 and PC 2 grouped by k-means clusters, k=4",
    xlabel="PC 1", 
    ylabel="PC 2",
    )

In [ ]:
k_plot₆ = scatter(convert(Matrix,df₁[:,2:8]) * 𝑉[:,1],
    convert(Matrix,df₁[:,2:8]) * 𝑉[:,2],
    marker_z = k_cluster₆.assignments,
    color =:rainbow,
    size=(800,800),
    markersize=8,
    markershape = gene_shapes,
    hover = df₁.ScaffoldID,
    legend=:none,
    title ="PC 1 and PC 2 grouped by k-means clusters, k=6",
    xlabel="PC 1", 
    ylabel="PC 2",
    )

In [ ]:
k_plot₇ = scatter(convert(Matrix,df₁[:,2:8]) * 𝑉[:,1],
    convert(Matrix,df₁[:,2:8]) * 𝑉[:,2],
    marker_z = k_cluster₇.assignments,
    color =:rainbow,
    size=(800,800),
    markersize=8,
    markershape = gene_shapes,
    hover = df₁.ScaffoldID,
    legend=:none,
    title ="PC 1 and PC 2 grouped by k-means clusters, k=7",
    xlabel="PC 1", 
    ylabel="PC 2",
    )